In [1]:
from insightface.app import FaceAnalysis
from insightface.model_zoo.arcface_onnx import ArcFaceONNX
from insightface.utils import face_align
import cv2
import numpy as np
from copy import deepcopy
from types import MethodType

def rtn_face_get(self, img, face):
    aimg = face_align.norm_crop(img, landmark=face.kps, image_size=self.input_size[0])
    #print(cv2.imwrite("aimg.png", aimg))
    face.embedding = self.get_feat(aimg).flatten()
    face.crop_face = aimg
    return face.embedding

ArcFaceONNX.get = rtn_face_get
app = FaceAnalysis(name="buffalo_l", providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))

/home/ubuntu/miniconda3/lib/python3.11/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
se

In [20]:
def crop_face(img, face_ratio=1.0):
    cv2_img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    faces = app.get(cv2_img)
    try:
        bbox = faces[0]['bbox']

        # get largest possible square around center point
        w,h = img.width, img.height
        box_w = bbox[2] - bbox[0]
        box_h = bbox[3] - bbox[1]
        if box_w < box_h:
            diff = box_h - box_w
            bbox[0] -= diff // 2
            bbox[2] += diff // 2
        else:
            diff = box_w - box_h
            bbox[1] -= diff // 2
            bbox[3] += diff // 2
        dist_to_left = bbox[0]
        dist_to_right = w - bbox[2]
        dist_to_top = bbox[1]
        dist_to_bottom = h - bbox[3]
        min_dist = min(dist_to_left, dist_to_right, dist_to_top, dist_to_bottom)
        min_dist = int(min_dist * face_ratio) - 1
        bbox = [bbox[0]-min_dist, bbox[1]-min_dist, bbox[2]+min_dist, bbox[3]+min_dist]
        face_image = img.crop(bbox)

        return face_image
    except:
        return img

In [31]:
import os
from tqdm import tqdm

base_path = "/home/ubuntu/AutoLoRADiscovery/people"
people_paths = [os.path.join(base_path, person) for person in os.listdir(base_path)]
for person_path in tqdm(people_paths):
    print(person_path)
    image_paths = [os.path.join(person_path, image_path) for image_path in os.listdir(person_path) if not ".txt" in image_path]
    for img_path in image_paths:
        img = Image.open(img_path).convert("RGB")
        cropped = crop_face(img)
        cropped.save(img_path)

In [13]:
path = "/home/ubuntu/AutoLoRADiscovery/common/lora_outputs/lora-Al Pacino/lora_layers.pth"

import torch

state_dict = torch.load(path)

In [14]:
state_dict

{'unet': {'down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_out.0.lora_down.weight': tensor([[-0.0402,  0.0740,  0.0590,  ..., -0.1050, -0.1954,  0.0289],
          [ 0.0067,  0.0339, -0.1954,  ...,  0.1033,  0.0579, -0.0405],
          [-0.0329,  0.1669, -0.0197,  ..., -0.0321, -0.1505, -0.0500],
          ...,
          [ 0.1557, -0.0366, -0.1391,  ..., -0.0465, -0.2156,  0.1907],
          [ 0.0069,  0.0689, -0.0787,  ...,  0.1205, -0.0097, -0.0303],
          [ 0.0173, -0.0652,  0.0429,  ...,  0.0322,  0.1332,  0.0952]],
         device='cuda:0'),
  'down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_out.0.lora_up.weight': tensor([[ 1.8122e-04, -3.9716e-04,  7.3137e-04,  ..., -1.8999e-04,
           -6.7267e-05,  8.1118e-04],
          [ 2.2995e-03,  2.4589e-03, -2.9358e-03,  ...,  1.1967e-03,
           -1.2616e-03,  3.3545e-03],
          [ 2.0069e-03,  3.2892e-03, -2.2044e-03,  ...,  1.1277e-03,
           -2.0190e-03,  1.9251e-03],
          ...,
          [-2.488

In [35]:
import glob
import os

def aggregate_loras(path):
    files = glob.glob(f"{path}/**/checkpoint-1000", recursive=True)
    return files

In [36]:
files = aggregate_loras("/home/ubuntu/AutoLoRADiscovery/common/lora_outputs")

In [37]:
for file in files:
    os.remove(file)